In [ ]:
import MLJ
import DataFrames as DF
import CSV
import CloudMicrophysics.PreprocessAerosolData
using DataFramesMeta
using CairoMakie
using LaTeXStrings

In [ ]:
include("ReadAerosolDataset.jl")

In [ ]:
function plot_single_var_act_frac_profile(grid_pos, dataset_filename::String, x_axis_var::Symbol, mach_filenames, mach_labels; kwargs...)
    X_test, Y_test = read_aerosol_dataset(dataset_filename)
    ARG_act_frac = PreprocessAerosolData.get_ARG_act_frac(X_test)[:,1]
    PySDM_act_frac = Y_test
    x_range = X_test[!, x_axis_var]
    ax = Axis(grid_pos; ylabel="Mode 1 activation fracion", kwargs...)
    lines!(ax, x_range, PySDM_act_frac, color=:black, label="PySDM")
    lines!(ax, x_range, ARG_act_frac, label="ARG")
    for (mach_filename, mach_label) in zip(mach_filenames, mach_labels)
        mach = MLJ.machine(mach_filename)
        pred_act_frac = MLJ.predict(mach, X_test)
        lines!(ax, x_range, pred_act_frac, label=mach_label)
    end
    return ax
end

In [ ]:
mach_filenames = ["emulators/1modal_nn_machine1.jls"]
mach_labels = ["NN model"]
fig = Figure(resolution=(1000, 1000))
ax = plot_single_var_act_frac_profile(fig[1,1], "datasets/single_var_profiles/Silva_Ma_Fig7_mode_N.csv", :mode_1_N, mach_filenames, mach_labels, xlabel="Aerosol number concentration (m^-3)", xscale=log10)
plot_single_var_act_frac_profile(fig[1,2], "datasets/single_var_profiles/Silva_Ma_Fig7_mode_mean.csv", :mode_1_mean, mach_filenames, mach_labels, xlabel="Aerosol mean dry radius (m)", xscale=log10)
plot_single_var_act_frac_profile(fig[2,1], "datasets/single_var_profiles/Silva_Ma_Fig7_velocity.csv", :velocity, mach_filenames, mach_labels, xlabel="Updraft velocity (m/s)", xscale=log10)
plot_single_var_act_frac_profile(fig[2,2], "datasets/single_var_profiles/Silva_Ma_Fig7_kappa.csv", :mode_1_kappa, mach_filenames, xlabel="Aerosol hygroscopicity", mach_labels)
Legend(fig[1,3], ax)
save("plots/single_var_profiles_1modal.pdf", fig)
fig

In [ ]:
mach_filenames = ["emulators/2modal_nn_machine1.jls"]
mach_labels = ["NN model"]
fig = Figure(resolution=(1200, 500))
ax = plot_single_var_act_frac_profile(fig[1,1], "datasets/single_var_profiles/ARG2000_Fig1_mode_2_N.csv", :mode_2_N, mach_filenames, mach_labels, xlabel="Mode 2 aerosol number concentration (m^-3)")
plot_single_var_act_frac_profile(fig[1,2], "datasets/single_var_profiles/ARG2000_Fig4_mode_2_mean.csv", :mode_2_mean, mach_filenames, mach_labels, xlabel="Mode 2 aerosol mean dry radius (m)", xscale=log10)
plot_single_var_act_frac_profile(fig[1,3], "datasets/single_var_profiles/ARG2000_Fig7_velocity.csv", :velocity, mach_filenames, mach_labels, xlabel="Updraft velocity (m/s)", xscale=log10)
Legend(fig[1,4], ax)
save("plots/single_var_profiles_2modal.pdf", fig)
fig